## Tab+LatLong Companies from BikesharingWorldMap

In [216]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
from datetime import datetime
#import import_ipynb


## Vytvoření 1. DataFramu - připojení ke zdrojovému kódu html BikesharingWorldMap

In [154]:
#import LatLong_Tab_Mix_BikeshareMap_BikesharingWorldMap

In [155]:
# preparing URL – request GET and RESPONSE into soup
url = "https://bikesharingworldmap.com/"
response = requests.get(url)
print(response)
print(type(response.text))
html = response.text
soup = BeautifulSoup(html, "html.parser")

<Response [200]>
<class 'str'>


In [156]:
# preparing data for cleaning

#soup.find("div", {"id": "legend"})
div_lst = soup.find_all("script")
type(div_lst)

bs4.element.ResultSet

In [227]:
div_lst

[<script async="" defer="" src="https://apis.google.com/js/platform.js"></script>,
 <script src="lib/jquery-3.5.1.js" type="text/javascript"></script>,
 <script src="https://lib.oomap.co.uk/jquery-ui-1.12.1/jquery-ui.js" type="text/javascript"></script>,
 <script src="https://lib.oomap.co.uk/openlayers/v6.3.1-dist/ol.js" type="text/javascript"></script>,
 <script src="main.js?t=1645216972" type="text/javascript"></script>,
 <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-424605-15"></script>,
 <script>
 	  window.dataLayer = window.dataLayer || [];
 	  function gtag(){dataLayer.push(arguments);}
 	  gtag('js', new Date());
 
 	  gtag('config', 'UA-424605-15');
 	</script>,
 <script async="" crossorigin="anonymous" defer="" nonce="WJ4KQMvu" src="https://connect.facebook.net/en_GB/sdk.js#xfbml=1&amp;version=v7.0&amp;appId=114474805306975&amp;autoLogAppEvents=1"></script>,
 <script type="text/javascript">var systems = {"success":true,"message":"","result":[["acoruna",

In [7]:
# zjištění indexů v listu pro daný tag
# for idx, obj in enumerate(div_lst):
#     print(idx, obj)

In [176]:
# čištění dat

unCleaned_data_0 = soup.find_all("script")[8].text


replacements = [
    (r"\\r\\n",""),
    (r"\\","")
]

for old, new in replacements:
    unCleaned_data_1 = re.sub(old, new, unCleaned_data_0)

#unCleaned_data_1 = re.sub(r"\\r\\n","",unCleaned_data_0)
#unCleaned_data_2 = re.sub(r"\\","",unCleaned_data_1)
len(unCleaned_data_2)

cleaned_data = unCleaned_data_2[52:278643]

In [177]:
cleaned_data

'[["acoruna","A Coruu00f1a, Spain - Bicicoruu00f1a",1,-8.39626,43.3723,163,2009,null],["aachen","Aachen, Germany - Call a Bike",3,6.092,50.7692,15,2011,2014],["aachen_velocity","Aachen, Germany - Velocity Aachen",1,6.0672,50.7789,300,2016,null],["aalborg","Aalborg, Denmark - City Bikes",3,9.92134,57.0437,170,2009,2014],["aalborg_donkey","Aalborg, Denmark - Donkey Republic",1,9.92116,57.0503,72,2017,null],["aarhus","Aarhus, Denmark - Aarhus Bycykel",3,10.2118,56.157,450,2005,2020],["aarhus_donkey","Aarhus, Denmark - Donkey Republic",1,10.2083,56.1575,444,2018,null],["aberdeen","Aberdeen, United Kingdom - Big Issue Sharebike",6,-2.10114,57.1541,450,null,null],["abudhabi","Abu Dhabi, United Arab Emirates - ADCB Bikeshare",3,54.6144,24.4984,350,2014,2020],["abudhabi_careem","Abu Dhabi, United Arab Emirates - Careem BIKE",1,54.4465,24.4342,300,2021,null],["acquaviva","Acquaviva delle Fonti, Italy - BiciViva",3,16.8436,40.8966,70,2012,2014],["adelaide_airbike","Adelaide, Australia - AirBike"

In [178]:
# převedení do 2D listu

cleaned_data_lst = json.loads(cleaned_data)
len(cleaned_data_lst)
#print(cleaned_data_lst)

3314

In [151]:
#cleaned_data_lst

[['acoruna',
  'A Coruu00f1a, Spain - Bicicoruu00f1a',
  1,
  -8.39626,
  43.3723,
  163,
  2009,
  None],
 ['aachen',
  'Aachen, Germany - Call a Bike',
  3,
  6.092,
  50.7692,
  15,
  2011,
  2014],
 ['aachen_velocity',
  'Aachen, Germany - Velocity Aachen',
  1,
  6.0672,
  50.7789,
  300,
  2016,
  None],
 ['aalborg',
  'Aalborg, Denmark - City Bikes',
  3,
  9.92134,
  57.0437,
  170,
  2009,
  2014],
 ['aalborg_donkey',
  'Aalborg, Denmark - Donkey Republic',
  1,
  9.92116,
  57.0503,
  72,
  2017,
  None],
 ['aarhus',
  'Aarhus, Denmark - Aarhus Bycykel',
  3,
  10.2118,
  56.157,
  450,
  2005,
  2020],
 ['aarhus_donkey',
  'Aarhus, Denmark - Donkey Republic',
  1,
  10.2083,
  56.1575,
  444,
  2018,
  None],
 ['aberdeen',
  'Aberdeen, United Kingdom - Big Issue Sharebike',
  6,
  -2.10114,
  57.1541,
  450,
  None,
  None],
 ['abudhabi',
  'Abu Dhabi, United Arab Emirates - ADCB Bikeshare',
  3,
  54.6144,
  24.4984,
  350,
  2014,
  2020],
 ['abudhabi_careem',
  'Abu Dhabi

In [179]:
name_cols = ["id", "name_station","num_of_stations", "lon", "lat", "num_of_bikes", "launch", "terminated"]

In [180]:
df1 = pd.DataFrame(cleaned_data_lst, columns=name_cols)

In [181]:
df1.set_index('id', inplace=True)

In [193]:
df1.shape

(3314, 7)

In [237]:
#df1.to_excel("df1_Company_Points_bikesharingworldmap.xlsx")

## Vytvoření 2. DataFramu - točení IDs v requestu

In [124]:
def point_info(ad_id):
    
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
        'sec-ch-ua-platform': '"Windows"',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://bikesharingworldmap.com/',
        'Accept-Language': 'cs-CZ,cs;q=0.9',
    }
    
    params = (
        ('system', f'{ad_id}'),
    )

    response = requests.get('https://bikesharingworldmap.com/getsystem.php', headers=headers, params=params)
    return response.json()

In [125]:
[i for lst in cleaned_data_lst for i in lst]

['acoruna',
 'A Coruu00f1a, Spain - Bicicoruu00f1a',
 1,
 -8.39626,
 43.3723,
 163,
 2009,
 None,
 'aachen',
 'Aachen, Germany - Call a Bike',
 3,
 6.092,
 50.7692,
 15,
 2011,
 2014,
 'aachen_velocity',
 'Aachen, Germany - Velocity Aachen',
 1,
 6.0672,
 50.7789,
 300,
 2016,
 None,
 'aalborg',
 'Aalborg, Denmark - City Bikes',
 3,
 9.92134,
 57.0437,
 170,
 2009,
 2014,
 'aalborg_donkey',
 'Aalborg, Denmark - Donkey Republic',
 1,
 9.92116,
 57.0503,
 72,
 2017,
 None,
 'aarhus',
 'Aarhus, Denmark - Aarhus Bycykel',
 3,
 10.2118,
 56.157,
 450,
 2005,
 2020,
 'aarhus_donkey',
 'Aarhus, Denmark - Donkey Republic',
 1,
 10.2083,
 56.1575,
 444,
 2018,
 None,
 'aberdeen',
 'Aberdeen, United Kingdom - Big Issue Sharebike',
 6,
 -2.10114,
 57.1541,
 450,
 None,
 None,
 'abudhabi',
 'Abu Dhabi, United Arab Emirates - ADCB Bikeshare',
 3,
 54.6144,
 24.4984,
 350,
 2014,
 2020,
 'abudhabi_careem',
 'Abu Dhabi, United Arab Emirates - Careem BIKE',
 1,
 54.4465,
 24.4342,
 300,
 2021,
 None,


In [126]:
# Extrakce IDs
"""
for lst in cleaned_data_lst:
    for i in lst:
        print(i)
        
# list comprehension
[i for lst in cleaned_data_lst for i in lst]
"""

ids_lst = [i[0] for i in cleaned_data_lst]
len(ids_lst)
ids_lst[0:10]

['acoruna',
 'aachen',
 'aachen_velocity',
 'aalborg',
 'aalborg_donkey',
 'aarhus',
 'aarhus_donkey',
 'aberdeen',
 'abudhabi',
 'abudhabi_careem']

In [127]:
#vytvoření listu pro informace ke stanicím a následné jeho plnění slovníky s infem o bodech
datetime.now().strftime("%d/%m/%Y %H:%M:%S")

point_info_list = []
for _id in ids_lst:
    print(_id)
    info_pts = point_info(_id)
    point_info_list.append(info_pts)
    
datetime.now().strftime("%d/%m/%Y %H:%M:%S")

acoruna
aachen
aachen_velocity
aalborg
aalborg_donkey
aarhus
aarhus_donkey
aberdeen
abudhabi
abudhabi_careem
acquaviva
adelaide_airbike
adelaide_freebikes
adelaide
adelaide_obike
adelaide_ofo
agadir
agen
agrigento
agrinio
aguascalientes
agueda
ahmedabad
aigio
aigle
aix
aksu
aktau
alameda
alba
alba_eazymov
albacete_albabici
albacete
albano
albany
alboraya
albuquerque
alcala
alcazar
alessandria
algeciras
alghero
alhama
alicante
aljaraque
allentown
almansa
almaty
almeirim
almendralejo
almeria
almese
almetyevsk
alpharetta
alpignano
altea
amboise
amherst
amman
amorebieta
amrum
amsterdam_cargoroo
amsterdam_donkeyre
amsterdam_flickbike
amsterdam_hellobike
amsterdam_obike
amsterdam_urbee
amsterdam
amstetten
anaheim
anda
andorra
andria
angelholm
angers_citiz
angers_indigoweel
angers_ponybikes
angers
anji
ankang_hightech
ankang
ankara
annarbor
annapolis
anniston
anqing
anqiu
ansan
anshun
antalya
antequera
antibes
antwerp
anxi
anyang
aojiang
aracaju
aranda
aranjuez
arcata
arcata_tandem
archea
are

In [128]:
# očištění výsledného slovníku a připojení očištěných dat do nového listu
clean_pt_lst = []
for i in point_info_list:
    for k, v in i.items():
        if k == "result":
            clean_pt_lst.append(v)
            
# stejný zápis jen pomocí list comprehension
clean_pt_lst = [v for i in point_info_list for k, v in i.items() if k == "result"]

In [229]:
clean_pt_lst

[['acoruna',
  'A Coruña - Bicicoruña',
  '',
  '163',
  '23',
  None,
  None,
  '1',
  'http://www.bicicoruna.es/',
  '',
  '0',
  '2009-01-01',
  None,
  'https://citybik.es/map/bicicorunha',
  'The Municipal Housing, Services and Activities Company',
  'EMVSA',
  '',
  '',
  ''],
 ['aachen',
  'Aachen - Call a Bike',
  '',
  '15',
  '3',
  '15',
  None,
  '3',
  '',
  '',
  '0',
  '2011-08-16',
  '2014-01-01',
  '',
  'DB Rent',
  '',
  '',
  '',
  ''],
 ['aachen_velocity',
  'Aachen - Velocity Aachen',
  '',
  '300',
  '50',
  '300',
  None,
  None,
  'https://velocity-aachen.de/',
  '',
  '0',
  '2016-11-16',
  None,
  '',
  'Velocity Aachen',
  '',
  '',
  '',
  ''],
 ['aalborg',
  'Aalborg - City Bikes',
  '',
  '170',
  '21',
  None,
  None,
  '1',
  '',
  '',
  '0',
  '2009-06-01',
  '2014-10-01',
  '',
  'City of Aalborg',
  '',
  '',
  '',
  ''],
 ['aalborg_donkey',
  'Aalborg - Donkey Republic',
  '',
  '72',
  '37',
  None,
  None,
  '3',
  'https://www.donkey.bike/cities/

In [196]:
name_cols_2 = ["id", "city_company","a", "bikes", "stations", "inc_pedelecs", "b", "c", "web_company", "d", "working", "date_launch", "date_terminated", "e", "operator", "equipment", "f", "pic", "g"]

In [120]:
len(name_cols_2)

19

In [197]:
df2 = pd.DataFrame(clean_pt_lst, columns=name_cols_2)

In [198]:
df2.set_index("id", inplace=True)

In [199]:
df2.shape

(3292, 18)

In [200]:
df1.shape[0]-df2.shape[0]

22

In [236]:
#df2.to_excel("df2_Company_Points_bikesharingworldmap.xlsx")

- Joining DF1 a DF2

In [190]:
pd.concat([df1, df2], join="inner", axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [201]:
df3 = df1.join(df2, rsuffix="_sec", how="left")

In [203]:
df3

,name_station,num_of_stations,lon,lat,num_of_bikes,launch,terminated,city_company,a,bikes,...,d,working,date_launch,date_terminated,e,operator,equipment,f,pic,g
id,,,,,,,,,,,,,,,,,,,,,
Chilecito,"Chilecito, Argentina - Rioja en Bici",2,-67.49520,-29.1598,NaN,NaN,NaN,Chilecito - Rioja en Bici,,None,...,,None,None,None,,,,,,
Cycle!,"Glens Falls, Queensbury and Lake George, Unite...",1,-73.64500,43.3094,20.0,NaN,NaN,"Glens Falls, Queensbury and Lake George -",,20,...,,208614,None,None,,CDPHP,,,https://poststar.com/news/local/bike-sharing-p...,
Gizycko,"Gizycko, Poland - GROM",4,21.76840,54.0363,20.0,2018.0,NaN,Gizycko - GROM,,20,...,,134135,2018-07-23,None,,ROOVEE,ROOVEE,,https://mazury24.eu/aktualnosci/rowerem-miejsk...,
Kumagaya,"Kumagaya, Japan - Wild Knights Cycle",1,139.37400,36.1624,100.0,2021.0,NaN,Kumagaya - Wild Knights Cycle,,100,...,,1002160,2021-09-16,None,,,,,https://kumagaya.keizai.biz/headline/957/,
Maebashi,"Maebashi, Japan - Akagi cogbe",1,139.12300,36.4346,NaN,2021.0,NaN,Maebashi - Akagi cogbe,,None,...,,101700,2021-10-23,None,,Ecobike,,,https://prtimes.jp/main/html/rd/p/000000008.00...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zurich,"Zu00fcrich, Switzerland - Zu00fcri Velo",1,8.53822,47.3823,350.0,2018.0,NaN,Zürich - Züri Velo,,350,...,,0,2018-04-06,None,,Publibike,Publibike,,,
zurich_lime,"Zu00fcrich, Switzerland - Lime",1,8.54105,47.3720,560.0,2018.0,NaN,Zürich - Lime,,560,...,,0,2018-01-04,None,,Lime,Limebike,,,
zurich_obike,"Zu00fcrich, Switzerland - oBike",3,8.54084,47.3676,700.0,2017.0,2018.0,Zürich - oBike,,700,...,,0,2017-07-07,2018-06-29,,,,,,


In [204]:
df3.columns

Index(['name_station', 'num_of_stations', 'lon', 'lat', 'num_of_bikes',
       'launch', 'terminated', 'city_company', 'a', 'bikes', 'stations',
       'inc_pedelecs', 'b', 'c', 'web_company', 'd', 'working', 'date_launch',
       'date_terminated', 'e', 'operator', 'equipment', 'f', 'pic', 'g'],
      dtype='object')

In [215]:
df3.loc[:, ["lon", "lat"]].dtypes

lon    float64
lat    float64
dtype: object

In [235]:
#df3.to_excel("df3_Company_Points_bikesharingworldmap.xlsx")

In [230]:
geometry = [Point(LonLat) for LonLat in zip(df3.lon, df3.lat)]
crs = {'init': 'epsg:4326'}
geo_df = GeoDataFrame(df3, crs=crs, geometry=geometry)

#geo_df.to_file(driver='ESRI Shapefile', filename='Company_Points_bikesharingworldmap.shp')

C:\Users\Milan\anaconda3\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)
